In [10]:
import pandas as pd
import json
import pprint

In [17]:
data = json.load(open("./odis_json/1029001.json"))
df = pd.json_normalize(data['innings'][0]["overs"])
print(df)

    over                                         deliveries
0      0  [{'batter': 'EC Joyce', 'bowler': 'DJ Willey',...
1      1  [{'batter': 'EC Joyce', 'bowler': 'MA Wood', '...
2      2  [{'batter': 'PR Stirling', 'bowler': 'DJ Wille...
3      3  [{'batter': 'EC Joyce', 'bowler': 'MA Wood', '...
4      4  [{'batter': 'PR Stirling', 'bowler': 'DJ Wille...
5      5  [{'batter': 'EC Joyce', 'bowler': 'MA Wood', '...
6      6  [{'batter': 'WTS Porterfield', 'bowler': 'DJ W...
7      7  [{'batter': 'WTS Porterfield', 'bowler': 'MA W...
8      8  [{'batter': 'WTS Porterfield', 'bowler': 'DJ W...
9      9  [{'batter': 'A Balbirnie', 'bowler': 'MA Wood'...
10    10  [{'batter': 'A Balbirnie', 'bowler': 'DJ Wille...
11    11  [{'batter': 'A Balbirnie', 'bowler': 'JT Ball'...
12    12  [{'batter': 'A Balbirnie', 'bowler': 'LE Plunk...
13    13  [{'batter': 'WTS Porterfield', 'bowler': 'JT B...
14    14  [{'batter': 'A Balbirnie', 'bowler': 'LE Plunk...
15    15  [{'batter': 'A Balbirnie', 'bo

In [14]:
csv = df.to_csv()
pprint.pprint(csv)

(',team,overs,powerplays,target.overs,target.runs\r\n'
 '0,Ireland,"[{\'over\': 0, \'deliveries\': [{\'batter\': \'EC Joyce\', '
 "'bowler': 'DJ Willey', 'non_striker': 'PR Stirling', 'runs': {'batter': 1, "
 "'extras': 0, 'total': 1}}, {'batter': 'PR Stirling', 'bowler': 'DJ Willey', "
 "'non_striker': 'EC Joyce', 'runs': {'batter': 0, 'extras': 0, 'total': 0}}, "
 "{'batter': 'PR Stirling', 'bowler': 'DJ Willey', 'extras': {'legbyes': 1}, "
 "'non_striker': 'EC Joyce', 'runs': {'batter': 0, 'extras': 1, 'total': 1}}, "
 "{'batter': 'EC Joyce', 'bowler': 'DJ Willey', 'non_striker': 'PR Stirling', "
 "'runs': {'batter': 2, 'extras': 0, 'total': 2}}, {'batter': 'EC Joyce', "
 "'bowler': 'DJ Willey', 'non_striker': 'PR Stirling', 'runs': {'batter': 1, "
 "'extras': 0, 'total': 1}}, {'batter': 'PR Stirling', 'bowler': 'DJ Willey', "
 "'non_striker': 'EC Joyce', 'runs': {'batter': 0, 'extras': 0, 'total': "
 "0}}]}, {'over': 1, 'deliveries': [{'batter': 'EC Joyce', 'bowler': 'MA "
 "Wood',

In [7]:
import pandas as pd
import json

# Load the JSON file
file_path = './odis_json/64819.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Prepare a list to collect all deliveries data
deliveries_data = []

# Iterate through each inning in the "innings" list
for inning in data.get('innings', []):
    team = inning.get('team', '')
    for over in inning.get('overs', []):
        over_number = over.get('over', None)
        for delivery in over.get('deliveries', []):
            # Extract relevant fields from each delivery
            extras = []
            if delivery.get('extras'):
                for i in delivery['extras'].keys():
                    extra_type = ''
                    extra_run = 0
                    extra_type = i
                    extra_run = delivery['extras'][i]
                    extras.append((extra_type, extra_run))
            delivery_info = {
                'team': team,
                'over': over_number,
                'batter': delivery.get('batter', ''),
                'bowler': delivery.get('bowler', ''),
                'non_striker': delivery.get('non_striker', ''),
                'batter_runs': delivery.get('runs', {}).get('batter', 0),
                'extras_runs': delivery.get('runs', {}).get('extras', 0),
                'total_runs': delivery.get('runs', {}).get('total', 0)
            }
            for i in extras:
                delivery_info[f'extra_{i[0]}_runs'] = i[1]
            # Append to the list
            deliveries_data.append(delivery_info)

# Convert the list to a DataFrame
df_deliveries = pd.DataFrame(deliveries_data)

# Save the DataFrame to a CSV file
output_csv_path = 'standardized_innings_output.csv'
df_deliveries.fillna(0, inplace=True)
df_deliveries.to_csv(output_csv_path, index=False)

print(f"Standardized innings CSV saved to {output_csv_path}")


Standardized innings CSV saved to standardized_innings_output.csv


In [21]:
import pandas as pd
import json

# Load the JSON file
file_path = './odis_json/64819.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Prepare a list to collect all deliveries data
deliveries_data = []

# Variables to keep track of previous delivery's batter and non-striker
previous_batter = None
previous_non_striker = None

# Iterate through each inning in the "innings" list
for inning in data.get('innings', []):
    team = inning.get('team', '')
    for over in inning.get('overs', []):
        over_number = over.get('over', None)
        for delivery in over.get('deliveries', []):
            # Extract relevant fields from each delivery
            extras = []
            if delivery.get('extras'):
                for i in delivery['extras'].keys():
                    extra_type = ''
                    extra_run = 0
                    extra_type = i
                    extra_run = delivery['extras'][i]
                    extras.append((extra_type, extra_run))
            
            # Initialize wicket to 0
            wicket = 0
            
            # Current delivery's batter and non-striker
            current_batter = delivery.get('batter', '')
            current_non_striker = delivery.get('non_striker', '')

            # Check if there's a change in batter or non-striker (indicating a wicket)
            if previous_batter and previous_non_striker:
                if (current_batter != previous_batter and current_batter != previous_non_striker) or \
                   (current_non_striker != previous_non_striker and current_non_striker != previous_batter):
                    wicket = 1

            # Update the previous batter and non-striker
            previous_batter = current_batter
            previous_non_striker = current_non_striker
            delivery_info = {
                'team': team,
                'over': over_number,
                'batter': current_batter,
                'bowler': delivery.get('bowler', ''),
                'non_striker': current_non_striker,
                'batter_runs': delivery.get('runs', {}).get('batter', 0),
                'extras_runs': delivery.get('runs', {}).get('extras', 0),
                'total_runs': delivery.get('runs', {}).get('total', 0),
            }

            for i in extras:
                delivery_info[f'extra_{i[0]}_runs'] = i[1]

            deliveries_data.append(delivery_info)
                    
            if len(deliveries_data) > 1:
                deliveries_data[len(deliveries_data) - 2]['wicket'] = wicket 
            else:
                deliveries_data[len(deliveries_data) - 1]['wicket'] = wicket 
            # Append to the list

# Convert the list to a DataFrame
df_deliveries = pd.DataFrame(deliveries_data)

# Save the DataFrame to a CSV file
output_csv_path = 'standardized_innings_output.csv'
df_deliveries.fillna(0, inplace=True)
df_deliveries.to_csv(output_csv_path, index=False)

print(f"Standardized innings CSV saved to {output_csv_path}")


Standardized innings CSV saved to standardized_innings_output.csv


In [2]:
import pandas as pd
import json
import os

# Directory containing the JSON files
directory_path = './all_json/'

# Prepare a list to collect all deliveries data from all files
all_deliveries_data = []

# Get a list of all JSON files in the directory
json_files = [f for f in os.listdir(directory_path) if f.endswith('.json')]
total_files = len(json_files)

# Iterate through each JSON file in the directory
for idx, filename in enumerate(json_files):
    file_path = os.path.join(directory_path, filename)
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Extract the team names
    innings = data.get('innings', [])
    if len(innings) == 2:
        team_1 = innings[0].get('team', '')
        team_2 = innings[1].get('team', '')
    else:
        team_1 = innings[0].get('team', '')
        team_2 = "Unknown"

    # Variables to keep track of previous delivery's batter and non-striker
    previous_batter = None
    previous_non_striker = None

    # Iterate through each inning in the "innings" list
    for inning in innings:
        team = inning.get('team', '')
        opponent_team = team_2 if team == team_1 else team_1
        for over in inning.get('overs', []):
            over_number = over.get('over', None)
            for delivery in over.get('deliveries', []):
                # Extract relevant fields from each delivery
                extras = []
                if delivery.get('extras'):
                    for i in delivery['extras'].keys():
                        extra_type = ''
                        extra_run = 0
                        extra_type = i
                        extra_run = delivery['extras'][i]
                        extras.append((extra_type, extra_run))

                # Initialize wicket to 0
                wicket = 0

                # Current delivery's batter and non-striker
                current_batter = delivery.get('batter', '')
                current_non_striker = delivery.get('non_striker', '')

                # Check if there's a change in batter or non-striker (indicating a wicket)
                if previous_batter and previous_non_striker:
                    if (current_batter != previous_batter and current_batter != previous_non_striker) or \
                       (current_non_striker != previous_non_striker and current_non_striker != previous_batter):
                        wicket = 1

                # Update the previous batter and non-striker
                previous_batter = current_batter
                previous_non_striker = current_non_striker

                delivery_info = {
                    'team': team,
                    'opponent_team': opponent_team,  # Add opponent team field
                    'over': over_number,
                    'batter': current_batter,
                    'bowler': delivery.get('bowler', ''),
                    'non_striker': current_non_striker,
                    'batter_runs': delivery.get('runs', {}).get('batter', 0),
                    'extras_runs': delivery.get('runs', {}).get('extras', 0),
                    'total_runs': delivery.get('runs', {}).get('total', 0),
                    'file': filename  # Optional: Include the file name to identify the source
                }

                for i in extras:
                    delivery_info[f'extra_{i[0]}_runs'] = i[1]

                all_deliveries_data.append(delivery_info)

                if len(all_deliveries_data) > 1:
                    all_deliveries_data[len(all_deliveries_data) - 2]['wicket'] = wicket 
                else:
                    all_deliveries_data[len(all_deliveries_data) - 1]['wicket'] = wicket

    # Print progress
    print(f"Processed {idx + 1}/{total_files} files: {filename}")

# Convert the list to a DataFrame
df_all_deliveries = pd.DataFrame(all_deliveries_data)

# Save the DataFrame to a CSV file
output_csv_path = 'combined_standardized_innings_output_of_all_types.csv'
df_all_deliveries.fillna(0, inplace=True)
df_all_deliveries.to_csv(output_csv_path, index=False)

print(f"Combined standardized innings CSV saved to {output_csv_path}")


Processed 1/17558 files: 1000851.json
Processed 2/17558 files: 1000853.json
Processed 3/17558 files: 1000855.json
Processed 4/17558 files: 1000881.json
Processed 5/17558 files: 1000883.json
Processed 6/17558 files: 1000885.json
Processed 7/17558 files: 1000887.json
Processed 8/17558 files: 1000889.json
Processed 9/17558 files: 1000891.json
Processed 10/17558 files: 1000893.json
Processed 11/17558 files: 1000895.json
Processed 12/17558 files: 1001349.json
Processed 13/17558 files: 1001351.json
Processed 14/17558 files: 1001353.json
Processed 15/17558 files: 1001371.json
Processed 16/17558 files: 1001373.json
Processed 17/17558 files: 1001375.json
Processed 18/17558 files: 1002143.json
Processed 19/17558 files: 1002145.json
Processed 20/17558 files: 1002147.json
Processed 21/17558 files: 1002149.json
Processed 22/17558 files: 1002151.json
Processed 23/17558 files: 1002157.json
Processed 24/17558 files: 1003271.json
Processed 25/17558 files: 1003273.json
Processed 26/17558 files: 1003843.